In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from simpletransformers.classification import ClassificationModel, ClassificationArgs

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight

In [2]:
# Data with Throughput & WorkTime
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem_test.csv')
df = df.dropna()

In [3]:
y = df["Input.deception_quadrant"].copy()
X = df[["Input.deception_quadrant"]]
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
splits_generator = sss.split(X, y)

for train_idx, test_idx in splits_generator:
    indices_train = train_idx
    indices_test = test_idx

train = df.take(indices_train)
test = df.take(indices_test)

In [4]:
new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].to_numpy()
deception_class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train_deception),
                                                 y_train_deception)

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 1 1 1] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [5]:
X_train = train[['Input.full_text', 'Input.deception_quadrant' ]]
X_test = test[['Input.full_text', 'Input.deception_quadrant' ]]

In [6]:
X_train = X_train.rename(columns={'Input.full_text': 'text', 'Input.deception_quadrant': 'labels'})
X_test = X_test.rename(columns={'Input.full_text': 'text', 'Input.deception_quadrant': 'labels'})

In [7]:
X_train['labels'] = X_train["labels"].apply(lambda x : [x])

X_train

,text,labels
10,"'til death or end of beta do us part, then",[0]
5,It seems like there are a lot of ways that cou...,[1]
8,I am pretty conflicted about whether to guess ...,[1]
6,"Sorry Italy I've been away doing, um, German t...",[1]


In [8]:
# Model type get from : https://simpletransformers.ai/docs/classification-specifics/
# Model name get from huggingface 
# bert / bert-base-cased ; distilbert/ distilbert-base-cased ; xlnet / xlnet-base-cased; roberta / roberta-base-cased

model = ClassificationModel(
    model_type = 'xlnet', 
    model_name = 'xlnet-base-cased',
    weight=list(deception_class_weights),
    args={'reprocess_input_data': True,
          'use_cuda':True,
          'num_train_epochs':1,
          'learning_rate': 3e-5,
          'train_batch_size':64,
          'overwrite_output_dir': True
         }
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [9]:
model.train_model(X_train)

C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\torch\optim\lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(1, 0.8456170558929443)

In [10]:
predictions, raw_outputs = model.predict(X_test['text'].tolist())

In [11]:
X_test

,text,labels
7,"I don't think I'm ready to go for that idea, h...",Cassandra
9,I am going to take it literally and say even ...,Straightforward


In [12]:
predictions

array([1, 1], dtype=int64)